In [7]:
# Add python script files to the system env
import sys  
sys.path.insert(1, "..\\My Functions")
# 
import os
import numpy as np
import pandas as pd
import rasterio as rio
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
# Import the custom python script files
import my_Variogram
import my_RomanMetrics
import my_Variogram_Plot

In [8]:
# DISC\\Python
cwd = "..\\..\\"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [9]:
# Set g
g = 4.66

In [10]:
# Find the coordinate based on the name of the site in our .csv file
df_Site = pd.read_csv(cwd + "//Site.csv")
df_Site.iloc[50,:]

Number                                                        51
Site                                           Jolanda di Savoia
Serial Number (FLOX)                                         NaN
Country                                                      NaN
Location                                                     NaN
Latitude                                               44.874305
Longitude                                              11.979201
Priority                                                     NaN
Plant functional type (PFT)                                  NaN
Reference network                                            NaN
Number of components                                         NaN
Reduce ROI size (600 x 600 m)                                NaN
Move ROI                                                     NaN
Comments                                                     NaN
PRISMA Availability (900m Buffer ROI Check)                  NaN
PRISMA Earliest Date     

In [11]:
for index in range(50,51):
    temp_Site_Name = df_Site['Site'][index]
    print(f"{temp_Site_Name} starts!")
    list_std = []
    list_cv = []
    list_range = []
    list_nugget = []
    list_sill = []
    list_bins = []
    list_experimental_model = []
    list_fitted_model = []
    list_empiral_X = []
    list_empiral_Y = []
    if os.path.exists(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m.tif"):
        bool_600m = True
    else:
        bool_600m = False
    if bool_600m:
        # Set internal distance
        distances_Int = [30,30,100,300,300,300]
        # Set external distance
        distances_Ext = [100,300,300,600,900,1200]
    else:
        # Set internal distance
        distances_Int = [30,30,100,300,300]
        # Set external distance
        distances_Ext = [100,300,300,900,1200]
    # Get the unique distances so we can calculate the variogram first
    list_distances = list(set(distances_Int + distances_Ext))
    list_distances.sort()
    for i in range(len(list_distances)):
        if list_distances[i] == 600:
            if os.path.exists(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m Spostamento.tif"):
                print(f"{temp_Site_Name} has reduced and moved ROI!")
                img = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m Spostamento.tif")
            else:
                print(f"{temp_Site_Name} has reduced ROI!")
                img = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv 600m.tif")
        else:
                img = rio.open(cwd_Images_Processed + "\\" + temp_Site_Name + "\\NIRv " + str(list_distances[i]) + "m.tif")  
        band = img.read(1).reshape(-1)
        temp_std = my_Variogram.calc_STD(band)
        temp_cv = my_Variogram.calc_CV(band)
        coordinates = my_Variogram.get_Coordinates_from_Rio(img)
        temp_range, temp_nugget, temp_sill, temp_bins, temp_experimental_model, temp_fitted_model, temp_empiral_X, temp_empiral_Y = my_Variogram.calc_Variogram_Parameters(band,coordinates,list_distances[i])
        list_std.append(temp_std)
        list_cv.append(temp_cv)
        list_range.append(temp_range)
        list_nugget.append(temp_nugget)
        list_sill.append(temp_sill)
        list_bins.append(temp_bins)
        list_experimental_model.append(temp_experimental_model)
        list_fitted_model.append(temp_fitted_model)
        list_empiral_X.append(temp_empiral_X)
        list_empiral_Y.append(temp_empiral_Y)
        # print(f"Distance {list_distances[i]} has been calculated!")

    list_Rcv = []
    list_RAW_Score = []
    list_Rse = []
    list_Rst = []
    list_Rsv = []
    list_ST_Score = []
    for i in range(len(list_distances)):
        dist_Int = distances_Int[i]
        dist_Ext = distances_Ext[i]
        index_Int = list_distances.index(dist_Int)
        index_Ext = list_distances.index(dist_Ext)
        range_Int, nugget_Int, sill_Int, bins_Int, experimental_model_Int, fitted_model_Int = list_range[index_Int], list_nugget[index_Int], list_sill[index_Int], list_bins[index_Int], list_experimental_model[index_Int], list_fitted_model[index_Int]
        range_Ext, nugget_Ext, sill_Ext, bins_Ext, experimental_model_Ext, fitted_model_Ext = list_range[index_Ext], list_nugget[index_Ext], list_sill[index_Ext], list_bins[index_Ext], list_experimental_model[index_Ext], list_fitted_model[index_Ext]
        cv_Int = list_cv[index_Int]
        cv_Ext = list_cv[index_Ext]
        semivar_range_Exp_Int = my_Variogram.get_Semivar_Exp(dist_Int, bins_Int, experimental_model_Int)
        semivar_range_Exp_Ext = my_Variogram.get_Semivar_Exp(dist_Ext, bins_Ext, experimental_model_Ext)
        # Calculate Rcv
        Rcv = my_RomanMetrics.calc_Rcv(cv_Ext, cv_Int)
        # Calculate RAW Score
        RAW_Score = my_RomanMetrics.calc_RAWScore(Rcv)
        # Calculate Rse
        Rse = my_RomanMetrics.calc_Rse(g, range_Ext, range_Int)
        # Calculate Rst
        Rst = my_RomanMetrics.calc_Rst(my_RomanMetrics.calc_ST(semivar_range_Exp_Ext, nugget_Ext), my_RomanMetrics.calc_ST(semivar_range_Exp_Int, nugget_Int))
        # Calculate Rsv
        Rsv = my_RomanMetrics.calc_Rsv(my_RomanMetrics.calc_SV(fitted_model_Ext, range_Ext, nugget_Ext, sill_Ext), my_RomanMetrics.calc_SV(fitted_model_Int, range_Int, nugget_Int, sill_Int))
        # Calculate ST Score
        ST_Score = my_RomanMetrics.calc_STScore(Rcv, Rse, Rst, Rsv)
        list_Rcv.append(Rcv)
        list_RAW_Score.append(RAW_Score)
        list_Rse.append(Rse)
        list_Rst.append(Rst)
        list_Rsv.append(Rsv)
        list_ST_Score.append(ST_Score)
        print(f"Internal distance {dist_Int} and external distance {dist_Ext} has been computed!")
    # Make a dataframe to illustrate the numeric result of variograms
    df_Vario = pd.DataFrame({
        "Distance": list_distances,
        "STD": list_std,
        "CV": list_cv,
        "Range": list_range,
        "Sill": list_sill,
        "Nugget": list_nugget
    })
    df_Vario
    # Make a dataframe to illustrate the numeric result of roman metrics
    df_Roman = pd.DataFrame({
        "Internal Distance": distances_Int,
        "External Distance": distances_Ext,
        "Rcv": list_Rcv,
        "Rse": list_Rse,
        "Rst": list_Rst,
        "Rsv": list_Rsv,
        "RAW Score": list_RAW_Score,
        "ST Score": list_ST_Score
    })
    df_Roman
    df_Vario.to_csv(cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Vario.csv", index = False)
    df_Roman.to_csv(cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Roman.csv", index = False)
    my_Variogram_Plot.plot_Variogram(list_distances, list_bins, list_fitted_model, list_range, list_sill, list_nugget, list_empiral_X,list_empiral_Y, site_Name_Graph = temp_Site_Name, path_SaveFig = cwd_Images_Processed + "\\" + temp_Site_Name  + "\\Variogram.png", bool_SaveFig = True, bool_CloseFig = True)
    print(f"{temp_Site_Name} done!")

Jolanda di Savoia starts!


c:\ProgramData\anaconda3\envs\DISC\Lib\site-packages\skgstat\Variogram.py:1715: OptimizeWarning: Covariance of the parameters could not be estimated
  self.cof, self.cov = curve_fit(


Internal distance 30 and external distance 100 has been computed!
Internal distance 30 and external distance 300 has been computed!
Internal distance 100 and external distance 300 has been computed!
Internal distance 300 and external distance 900 has been computed!
Internal distance 300 and external distance 1200 has been computed!
Jolanda di Savoia done!
